In [1]:
import pandas as pd
import os
import csv

In [43]:
split = 'test'

In [46]:
if split == 'train':
    vectors_root_dir = 'vectors'
    labels_root_dir = 'labels'

    languages = ['de', 'en', 'fr', 'jp', 'zh', 'IPT']
    categories = ['apparel', 'home', 'musical_instruments', 'sports']

elif split == 'test':
    vectors_root_dir = 'vectors_test'
    labels_root_dir = 'labels_test'

    languages = ['de', 'en', 'fr', 'jp', 'zh']
    categories = ['apparel', 'home', 'musical_instruments', 'sports']

In [47]:
for language in languages:
    if split == 'train' and language == 'IPT':
        for language_ipt in languages + ['all']:
            
            if language_ipt == 'IPT':
                continue

            vectors_dir = os.path.join(vectors_root_dir, 'IPT', language_ipt)
            labels_dir = os.path.join(labels_root_dir, 'IPT', language_ipt)

            vectors_file = os.path.join(vectors_dir, 'vectors.txt')
            labels_file = os.path.join(labels_dir, 'labels.txt')
            df_vectors = pd.read_csv(vectors_file, header=None, sep='\t', names=['threadId', 'segmentId', 'text'])
            df_labels = pd.read_csv(labels_file, header=None, names=['LsegmentId', 'label'])
            # Drop threadId and segmentId columns
            df_vectors.drop(['threadId', 'segmentId'], axis=1, inplace=True)

            df = pd.concat([df_vectors, df_labels], axis=1)

            # Remove !@#NEXLP_DOCVECTOR_SEPARATOR#@! and !#nexlpEOL#! from the text
            df['text'] = df['text'].str.replace('!@#NEXLP_DOCVECTOR_SEPARATOR#@!', '')
            df['text'] = df['text'].str.replace('!#nexlpEOL#!', '')

            # Rearrange columns
            df = df[['LsegmentId', 'text', 'label']]

            # Rename column LsegmentId to Id
            df.rename(columns={'LsegmentId': 'ID'}, inplace=True)

            # Save as tsv
            train_dir = os.path.join('train', 'IPT', language_ipt)

            if not os.path.exists(train_dir):
                os.makedirs(train_dir)
            train_file = os.path.join(train_dir, 'train.tsv')
            df.to_csv(train_file, sep='\t', index=False, quoting=csv.QUOTE_NONE)
            
            print('Saved to {}'.format(train_file))
    
    else:
        for category in categories:
            if split == 'train':
                vectors_dir = os.path.join(vectors_root_dir, language, category, '100')
                labels_dir = os.path.join(labels_root_dir, language, category, '100')
            
            elif split == 'test':
                vectors_dir = os.path.join(vectors_root_dir, language, category, 'test_vectors')
                labels_dir = os.path.join(labels_root_dir, language, category, 'test_labels')

            vectors_file = os.path.join(vectors_dir, 'vectors.txt')
            labels_file = os.path.join(labels_dir, 'labels.txt')
            df_vectors = pd.read_csv(vectors_file, header=None, sep='\t', names=['threadId', 'segmentId', 'text'])
            df_labels = pd.read_csv(labels_file, header=None, names=['LsegmentId', 'label'])
            # Drop threadId and segmentId columns
            df_vectors.drop(['threadId', 'segmentId'], axis=1, inplace=True)

            df = pd.concat([df_vectors, df_labels], axis=1)
            
            # Remove !@#NEXLP_DOCVECTOR_SEPARATOR#@! and !#nexlpEOL#! from the text
            df['text'] = df['text'].str.replace('!@#NEXLP_DOCVECTOR_SEPARATOR#@!', '')
            df['text'] = df['text'].str.replace('!#nexlpEOL#!', '')

            # Rearrange columns
            df = df[['LsegmentId', 'text', 'label']]
            
            # Rename column LsegmentId to segmentId
            df.rename(columns={'LsegmentId': 'ID'}, inplace=True)

            # Save to train/{language}/{category}/train.tsv
            train_dir = os.path.join(split, language, category)
            if not os.path.exists(train_dir):
                os.makedirs(train_dir)
            train_file = os.path.join(train_dir, f'{split}.tsv')
            df.to_csv(train_file, sep='\t', index=False, quoting=csv.QUOTE_NONE)

        
            

In [24]:
df

,segmentId,text,label
0,65,Schiebermütze. kann ich nicht bewerten da zu g...,0
1,66,Panda Ganzkörper Tier-Kostüm. Ich bin sehr ent...,0
2,69,Unbrauchbar. Entweder waren die Handschuhe sch...,0
3,71,Schlechte Qualität. Nach dem zweiten Waschen k...,0
4,80,Hersteller verbieten bei Amazon!!!. Solch eine...,0
...,...,...,...
95,160607,Perfekt. Eine perfekte Brille. Gerade das Zube...,1
96,160609,Sehr schöner Mantel. Ich habe mir diesen Stepp...,1
97,160610,Jacke die hält was sie aussagt!. Größe passt! ...,1
98,160625,Endlich ist das Handy und Kleingeld aufgeräumt...,1
